In [3]:
%%capture
%load_ext autoreload
%autoreload 2
import numpy as np
from prfpy.stimulus import PRFStimulus2D
from prfpy.model import Iso2DGaussianModel, Norm_Iso2DGaussianModel
from prfpy.fit import Iso2DGaussianFitter, Norm_Iso2DGaussianFitter

This notebook shows a simple example of prfpy use, fitting a Gaussian and DN models and crossvalidating

# Creating stimulus object

In [4]:
screen_size_cm=40
screen_distance_cm=200
design_matrix=np.zeros((50,50,100))
design_matrix[:,20:30,40:50] = 1
TR=1
task_lengths=[100]
task_names=['test']
late_iso_dict=np.arange(40)
normalize_integral_dx=False

prf_stim = PRFStimulus2D(screen_size_cm=screen_size_cm,
                             screen_distance_cm=screen_distance_cm,
                             design_matrix=design_matrix,
                             TR=TR,
                             task_lengths=task_lengths,
                             task_names=task_names,
                             late_iso_dict=late_iso_dict,
                             normalize_integral_dx=normalize_integral_dx)

# Gaussian model fit

## Creating Gaussian model and fitter objects

In [5]:
filter_params={"first_modes_to_remove":3,
                         "last_modes_to_remove_percent":0,
                         "window_length":50,
                         "polyorder":3,
                         "highpass":True,
                         "add_mean":True}

data = 3*np.random.rand(50,100)+0.01*np.sum(design_matrix, axis=(0,1))-1.5

np.roll(data,6)

gg = Iso2DGaussianModel(stimulus=prf_stim,
                          hrf=[1,4,0],
                          filter_predictions=False,
                          filter_type='dc',
                          filter_params=filter_params,
                          normalize_RFs=False)

gf = Iso2DGaussianFitter(data=data, model=gg, n_jobs=8)

## Gaussian grid fit

In [6]:
 ecc_grid=np.linspace(0,10,10)
 polar_grid=np.linspace(-np.pi,np.pi,10)
 size_grid=np.linspace(1,10,10)
 verbose=False
 n_batches=8
 fixed_grid_baseline=0
 gauss_grid_bounds=[(0,1000)]
 hrf_1_grid=np.linspace(0,10,10)
 hrf_2_grid=np.linspace(0,0,1)

gf.grid_fit(ecc_grid=ecc_grid,
                polar_grid=polar_grid,
                size_grid=size_grid,
                verbose=verbose,
                n_batches=n_batches,
                fixed_grid_baseline=fixed_grid_baseline,
                grid_bounds=gauss_grid_bounds)#,
          # hrf_1_grid=hrf_1_grid,
         #  hrf_2_grid=hrf_2_grid)

/Users/marcoaqil/prfpy/prfpy/model.py:255: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  prediction_params = np.array([mu_x,
/Users/marcoaqil/prfpy/prfpy/model.py:314: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':


In [7]:
gf.gridsearch_params

array([[ 7.30872038e+00, -2.66015667e+00,  1.00000000e+01,
         8.15746374e-03,  0.00000000e+00,  4.00000000e+00,
         0.00000000e+00,  3.16694349e-01],
       [-0.00000000e+00, -0.00000000e+00,  1.00000000e+00,
         2.85725798e-02,  0.00000000e+00,  4.00000000e+00,
         0.00000000e+00,  2.58635074e-01],
       [ 1.66666667e+00, -2.88675135e+00,  8.00000000e+00,
         5.72532974e-03,  0.00000000e+00,  4.00000000e+00,
         0.00000000e+00,  2.64916778e-01],
       [-0.00000000e+00, -0.00000000e+00,  1.00000000e+00,
         3.19334455e-02,  0.00000000e+00,  4.00000000e+00,
         0.00000000e+00,  2.97376871e-01],
       [-0.00000000e+00, -0.00000000e+00,  1.00000000e+00,
         2.73207724e-02,  0.00000000e+00,  4.00000000e+00,
         0.00000000e+00,  2.89503306e-01],
       [-7.71769679e-01, -4.37692335e+00,  3.00000000e+00,
         1.29233673e-02,  0.00000000e+00,  4.00000000e+00,
         0.00000000e+00,  2.99249619e-01],
       [-0.00000000e+00, -0.000000

## Gaussian Iterative Fit

In [8]:
rsq_threshold=-10
verbose=True
gauss_bounds = [(-1.5*10, 1.5*10),  # x
                (-1.5*10, 1.5*10),  # y
                (0.1, 1.5*5),  # prf size
                (0, 1000),  # prf amplitude
                (0, 0)]  # bold baseline
gauss_bounds += [(0,10),(0,0)]
constraints=[]
xtol=1e-4
ftol=1e-4
gf.iterative_fit(rsq_threshold=rsq_threshold, verbose=verbose,
                         bounds=gauss_bounds,
                         constraints=constraints,
                             xtol=xtol,
                             ftol=xtol)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
/Users/marcoaqil/anaconda3/envs/prfpy_2021/lib/python3.9/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '
/Users/marcoaqil/anaconda3/envs/prfpy_2021/lib/python3.9/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '
/Users/marcoaqil/anaconda3/envs/prfpy_2021/lib/python3.9/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '
/Users/marcoaqil/anaconda3/envs/prfpy_2021/lib/python3.9/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change i

In [9]:
gf.iterative_search_params

array([[ 7.28114314e+00, -2.65230524e+00,  6.43537208e+00,
         1.02598632e-02,  0.00000000e+00,  6.94952911e+00,
         0.00000000e+00,  3.46221568e-01],
       [ 1.02195885e-03, -1.41346758e-04,  1.28216267e+00,
         1.54463734e-02,  0.00000000e+00,  7.53401004e+00,
         0.00000000e+00,  2.95269073e-01],
       [ 1.53167440e-01, -3.93593394e+00,  7.48527932e+00,
         4.00223935e-03,  0.00000000e+00,  9.26874401e+00,
         0.00000000e+00,  3.23768565e-01],
       [ 4.72932063e-09,  3.41814522e-04,  1.13757158e+00,
         2.30654607e-02,  0.00000000e+00,  5.73278013e+00,
         0.00000000e+00,  3.11163535e-01],
       [-3.06929560e-03, -7.00946882e-04,  1.04754704e+00,
         2.44198632e-02,  0.00000000e+00,  4.52701049e+00,
         0.00000000e+00,  2.90730886e-01],
       [-4.53615639e-01, -4.50704150e+00,  3.08487738e+00,
         8.63241314e-03,  0.00000000e+00,  8.89082589e+00,
         0.00000000e+00,  3.59943256e-01],
       [ 1.19098742e-03,  3.940568

## Crossvalidate

In [9]:
gf.crossvalidate_fit(test_data=test_data,
                     test_stimulus=test_stimulus,
                     single_hrf=single_hrf)

NameError: name 'test_data' is not defined

# DN model fit

## Create model and fitter objects

In [13]:
gg_norm = Norm_Iso2DGaussianModel(stimulus=prf_stim,
                                    hrf=[1,5,0],
                                    filter_predictions=False,
                                    filter_type='dc',
                                    filter_params=filter_params,                                       
                                    normalize_RFs=False)

gf_norm = Norm_Iso2DGaussianFitter(data=data,
                                   model=gg_norm,
                                   n_jobs=8,
                                   previous_gaussian_fitter=gf,
                                  use_previous_gaussian_fitter_hrf=False)

## DN model grid fit

In [14]:
norm_grid_bounds = [(0,1000),(0,1000)] #only prf amplitudes between 0 and 1000, only neural baseline values between 0 and 1000

gf_norm.grid_fit(surround_amplitude_grid=np.linspace(0,10,5),
             surround_size_grid=np.linspace(1,10,5),
             neural_baseline_grid=np.linspace(0,10,5),
             surround_baseline_grid=np.linspace(1,10,5),
             verbose=verbose,
             n_batches=8,
             rsq_threshold=rsq_threshold,
             fixed_grid_baseline=fixed_grid_baseline,
             grid_bounds=norm_grid_bounds,
             hrf_1_grid=np.linspace(0,10,5),
             hrf_2_grid=np.linspace(0,0,1))

Each batch contains approx. 7 voxels.


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:   11.3s
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:   11.7s remaining:   35.0s
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:   11.7s remaining:   19.5s
[Parallel(n_jobs=8)]: Done   4 out of   8 | elapsed:   11.7s remaining:   11.7s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:   11.7s remaining:    7.0s
[Parallel(n_jobs=8)]: Done   6 out of   8 | elapsed:   11.7s remaining:    3.9s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:   12.5s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:   12.5s finished


In [15]:
gf_norm.gridsearch_params

array([[ 7.28114314e+00, -2.65230524e+00,  6.43537208e+00,
         7.55350068e-02,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  7.75000000e+00,
         7.50000000e+00,  0.00000000e+00,  3.45695943e-01],
       [ 1.02195885e-03, -1.41346758e-04,  1.28216267e+00,
         5.03557995e-02,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  3.25000000e+00,
         7.50000000e+00,  0.00000000e+00,  2.95267373e-01],
       [ 1.53167440e-01, -3.93593394e+00,  7.48527932e+00,
         3.77072650e-03,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         1.00000000e+01,  0.00000000e+00,  3.23389947e-01],
       [ 4.72932063e-09,  3.41814522e-04,  1.13757158e+00,
         2.47207545e-02,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         5.00000000e+00,  0.00000000e+00,  3.09132993e-01],
       [-3.06929560e-03, -7.00946882e-04,  1.0475470

## DN Iterative Fit

In [56]:
norm_bounds =  [(-1.5*10, 1.5*10),  # x
                (-1.5*10, 1.5*10),  # y
                (0.1, 1.5*5),  # prf size
                (0, 1000),  # prf amplitude
                (0, 0),  # bold baseline
                (0, 1000),  # surround amplitude
                (0.1, 3*5),  # surround size
                (0, 1000),  # neural baseline
                (1e-6, 1000)]  # surround baseline
norm_bounds += [(0,10),(0,0)]
constraints_norm = []
gf_norm.iterative_fit(rsq_threshold=rsq_threshold, verbose=verbose,
                               bounds=norm_bounds,
                               constraints=constraints_norm,
                               xtol=xtol,
                               ftol=ftol)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
/Users/marcoaqil/prfpy/prfpy/model.py:545: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/Users/marcoaqil/prfpy/prfpy/model.py:545: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/Users/marcoaqil/prfpy/prfpy/model.py:545: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/Users/marcoaqil/prfpy/prfpy/model.py:545: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/Users/marcoaqil/prfpy/prfpy/model.py:545: FutureWarning: elementwise comparison failed; returning scalar instead, but in the fut

/Users/marcoaqil/prfpy/prfpy/model.py:545: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/Users/marcoaqil/prfpy/prfpy/model.py:545: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/Users/marcoaqil/prfpy/prfpy/model.py:545: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/Users/marcoaqil/prfpy/prfpy/model.py:545: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/Users/marcoaqil/prfpy/prfpy/model.py:545: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':
/User

In [57]:
gf.iterative_search_params

array([[ 1.94984664e-09,  0.00000000e+00,  2.00000001e+00,
         9.75012742e-03,  0.00000000e+00,  6.66649096e+00,
         0.00000000e+00,  2.76523271e-01],
       [-6.85026578e-08,  6.85026578e-08,  9.99999452e-01,
         2.29443754e-02,  0.00000000e+00,  5.54859555e+00,
         0.00000000e+00,  2.16892420e-01],
       [-4.89196156e-07,  0.00000000e+00,  9.99995597e-01,
         2.43040801e-02,  0.00000000e+00,  6.74929337e+00,
         0.00000000e+00,  3.10318638e-01],
       [ 0.00000000e+00,  0.00000000e+00,  2.00000000e+00,
         8.09554354e-03,  0.00000000e+00,  7.77753990e+00,
         0.00000000e+00,  1.59007620e-01],
       [-2.13240206e-06, -2.13297818e-06,  1.00009661e+00,
         2.53574705e-02,  0.00000000e+00,  6.14718068e+00,
         0.00000000e+00,  3.67713268e-01],
       [ 0.00000000e+00,  0.00000000e+00,  3.00000000e+00,
         7.53266212e-03,  0.00000000e+00,  6.66679977e+00,
         0.00000000e+00,  3.50206884e-01],
       [ 0.00000000e+00,  1.379520

In [70]:
gf_norm.iterative_search_params[:,-1] - gf.iterative_search_params[:,-1]

array([-1.28657472e-04, -1.51248740e-05,  1.24835351e-04,  1.13812708e-04,
       -4.10438616e-09,  8.25600276e-05,  6.32065384e-05,  1.00378443e-08,
       -5.26414468e-11, -6.03622295e-06])

## Crossvalidate

In [ ]:
gf_norm.crossvalidate_fit(test_data,
                        test_stimulus=test_stimulus,
                        single_hrf=single_hrf)